In [12]:
!pip install bcrypt
import bcrypt
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import base64
import requests
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import smtplib
from datetime import datetime, timedelta
import pytz
from pytz import country_timezones
import random
import hashlib
import getpass
import os
import json

# Load Google Sheets Credentials from GitHub Secrets
google_sheet_credentials = json.loads(os.getenv("GOOGLE_SHEET_CREDENTIALS"))
SHEET_ID = os.getenv("SHEET_ID")
RANGE_USERS = 'Users!A2:D'  # Users' data starts from row 2
RANGE_AVAILABILITY = 'Availability!A2:C'  # Availability data starts from row 2
RANGE_MATCHED = 'Matched!A2:F'  # Matched data starts from row 2
RANGE_CASES = 'Cases!A2:B'  # Case links in the Cases sheet

# Load Zoom and Email Credentials
SENDER_EMAIL = os.getenv("SENDER_EMAIL")
SENDER_PASSWORD = os.getenv("SENDER_PASSWORD")
ZOOM_CLIENT_ID = os.getenv("ZOOM_CLIENT_ID")
ZOOM_CLIENT_SECRET = os.getenv("ZOOM_CLIENT_SECRET")
ZOOM_ACCOUNT_ID = os.getenv("ZOOM_ACCOUNT_ID")
ZOOM_TOKEN_URL = os.getenv("ZOOM_TOKEN_URL")


# Authenticate Google Sheets API
def authenticate_google_sheets():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    creds = ServiceAccountCredentials.from_json_keyfile_name(google_sheet_credentials, SCOPES)
    client = gspread.authorize(creds)
    return client

# Get Zoom Access Token
def get_zoom_access_token():
    credentials = f"{ZOOM_CLIENT_ID}:{ZOOM_CLIENT_SECRET}"
    encoded_credentials = base64.b64encode(credentials.encode()).decode()

    headers = {
        "Authorization": f"Basic {encoded_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "account_credentials", "account_id": ZOOM_ACCOUNT_ID}

    response = requests.post(ZOOM_TOKEN_URL, headers=headers, data=data)

    if response.status_code == 200:
        return response.json().get("access_token")
    else:
        print(f"❌ Failed to obtain access token: {response.json()}")
        return None

# Create Zoom Meeting
def create_zoom_meeting():
    access_token = get_zoom_access_token()
    if not access_token:
        return None

    url = "https://api.zoom.us/v2/users/me/meetings"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    start_time = (datetime.utcnow() + timedelta(minutes=5)).isoformat()

    meeting_data = {
        "topic": "CaseBud Interview Meeting",
        "type": 2,
        "start_time": start_time,
        "duration": 40,
        "timezone": "UTC",
        "password": "12345",
        "settings": {
            "join_before_host": True,
            "host_video": False,
            "participant_video": True,
            "waiting_room": False  # No waiting room to avoid host start requirement
        }
    }

    response = requests.post(url, headers=headers, json=meeting_data)

    if response.status_code == 201:
        return response.json()["join_url"]
    else:
        print(f"❌ Failed to create Zoom meeting: {response.json()}")
        return None

# Send Email
def send_email(subject, body, to_email):
    EMAIL_ADDRESS = SENDER_EMAIL
    EMAIL_PASSWORD = SENDER_PASSWORD

    msg = MIMEMultipart()
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        server.sendmail(EMAIL_ADDRESS, to_email, msg.as_string())
        server.quit()
        print("✅ Email sent successfully to interviewer and interviewee")
    except Exception as e:
        print(f"❌ Failed to send email to {to_email}. Error: {e}")

# Choose Random Case
def choose_random_case():
    client = authenticate_google_sheets()
    sheet = client.open_by_key(SHEET_ID)
    cases_sheet = sheet.worksheet("Cases")
    cases = cases_sheet.get_all_records()
    if cases:
        case = random.choice(cases)
        return case['case_link']
    return None

from datetime import datetime, timedelta
import pytz

# Add Availability
def add_availability(email):
    client = authenticate_google_sheets()
    sheet = client.open_by_key(SHEET_ID)
    availability_sheet = sheet.worksheet("Availability")

    user_timezone = get_stored_timezone(email)  # Fetch stored timezone

    if not user_timezone:
        print("❌ Error: Time zone not found for your account.")
        return

    timezone_obj = pytz.timezone(user_timezone)  # Convert string to pytz timezone

    # Display the next 7 dates and ask the user to choose
    today = datetime.today()
    for i in range(7):
        future_date = today + timedelta(days=i)
        print(f"{i + 1}. {future_date.strftime('%A, %Y-%m-%d')}")

    date_choice = int(input("Choose the date (1-7): ")) - 1
    selected_date = (today + timedelta(days=date_choice)).strftime('%Y-%m-%d')

    print("Choose time slots between 8AM - 9PM, comma-separated (e.g., 9, 10, 14):")
    time_choices = input("Enter available hours in 24-hour format (e.g., 9, 10, 14): ").split(',')

    # Add each time slot separately
    for time_choice in time_choices:
        # Validate time
        if not (8 <= int(time_choice) <= 20):
            print("❌ Invalid time. Please select a time between 8AM and 9PM.")
            return

        time_slot = f"{time_choice.strip()}:00:00"

        # Convert the chosen time to the user's time zone
        local_time = datetime.strptime(f"{selected_date} {time_slot}", "%Y-%m-%d %H:%M:%S")

        # Localize the time with user's timezone
        localized_time = timezone_obj.localize(local_time)

        # Convert the chosen time to UTC
        utc_time = localized_time.astimezone(pytz.utc)

        # Store the date in the date column and only the UTC time in the time column
        availability_sheet.append_row([email, selected_date, utc_time.strftime('%H:%M:%S')])

    print("✅ Availability added successfully.")
    match_users(email)  # Run matching after adding availability

# Country-to-timezone selection using pytz
def get_user_timezone(email):
    country = input("Please enter your country (e.g., 'US', 'IN', 'GB'): ").strip().upper()

    # Get timezones for the given country code
    if country in country_timezones:
        timezones = country_timezones[country]
        print(f"Select your time zone in {country}:")

        # Display time zone options
        for i, timezone in enumerate(timezones, 1):
            print(f"{i}. {timezone}")

        timezone_choice = int(input("Enter the number corresponding to your time zone: ")) - 1
        selected_timezone = timezones[timezone_choice]

        print(f"You have selected {country} with time zone {selected_timezone}.")
        return selected_timezone
    else:
        print(f"Sorry, we do not have time zones for {country}. Please enter a valid country.")
        return None

def get_stored_timezone(email):
    client = authenticate_google_sheets()
    sheet = client.open_by_key(SHEET_ID)
    users_sheet = sheet.worksheet("Users")
    user_records = users_sheet.get_all_values()  # Fetch all user records
    for row in user_records:
        if row[0] == email:  # Assuming email is in the first column
            return row[2]  # Assuming timezone is stored in the third column
    return None  # If no matching email is found

def convert_utc_to_local(email, utc_datetime_str):
    """
    Converts a given UTC datetime string to the user's local timezone.

    Parameters:
    - email (str): User's email to fetch their timezone from the Users sheet.
    - utc_datetime_str (str): Datetime string in UTC format (e.g., '2025-03-09 14:30:00').

    Returns:
    - str: Localized datetime string in the format 'YYYY-MM-DD HH:MM AM/PM'.
    """
    client = authenticate_google_sheets()
    sheet = client.open_by_key(SHEET_ID)
    users_sheet = sheet.worksheet("Users")

    # Fetch all user data
    user_data = users_sheet.get_all_records()

    # Find the user's timezone
    user_timezone = None
    for user in user_data:
        if user['email'] == email:
            user_timezone = user.get('timezone')
            break

    if not user_timezone:
        print(f"⚠️ No timezone found for {email}, defaulting to UTC.")
        return utc_datetime_str  # Return original UTC time if no timezone found

    try:
        # Parse the UTC datetime string
        utc_dt = datetime.strptime(utc_datetime_str, "%Y-%m-%d %H:%M:%S")
        utc_dt = pytz.utc.localize(utc_dt)  # Set it as UTC

        # Convert to user's local time zone
        local_tz = pytz.timezone(user_timezone)
        local_dt = utc_dt.astimezone(local_tz)

        # Format the output
        return local_dt.strftime("%Y-%m-%d %I:%M %p")  # Example: '2025-03-09 09:30 AM'

    except Exception as e:
        print(f"⚠️ Error converting time: {e}")
        return utc_datetime_str  # Return original UTC time if an error occurs

def convert_utc_to_local_for_email(email, utc_datetime_str):
    user_timezone = get_stored_timezone(email)  # Fetch user's stored timezone

    if not user_timezone:
        print("❌ Error: Time zone not found for this account.")
        return None  # Return None if timezone isn't found

    # Convert string to datetime object (assuming UTC format)
    utc_datetime = datetime.strptime(utc_datetime_str, "%Y-%m-%d %H:%M:%S")
    utc_datetime = pytz.utc.localize(utc_datetime)  # Ensure it's timezone-aware

    # Convert to user's local timezone
    local_timezone = pytz.timezone(user_timezone)
    local_datetime = utc_datetime.astimezone(local_timezone)

    return local_datetime  # Now correctly returns a datetime object

def create_account(email, password):
    client = authenticate_google_sheets()
    sheet = client.open_by_key(SHEET_ID)
    users_sheet = sheet.worksheet("Users")

    # Check if the email already exists
    existing_users = users_sheet.get_all_records()
    if any(user['email'] == email for user in existing_users):
        print("❌ Account with this email already exists.")
        return False

    # Ask for the user's time zone if it's not provided yet
    user_timezone = get_user_timezone(email)

    # Hash the password
    salt = bcrypt.gensalt()
    hashed_password = bcrypt.hashpw(password.encode(), salt)

    # Convert hashed password to string (for JSON serialization)
    hashed_password_str = hashed_password.decode('utf-8')

    # Determine the role (Interviewer or Interviewee)
    interviewers = [user for user in existing_users if user['role'] == 'interviewer']
    interviewees = [user for user in existing_users if user['role'] == 'interviewee']

    if len(interviewers) > len(interviewees):
        role = 'interviewee'  # Assign interviewee if there are more interviewers
    else:
        role = 'interviewer'  # Assign interviewer if there are more interviewees or it's balanced

    # Store the new account with role and time zone
    users_sheet.append_row([email, hashed_password_str, user_timezone, role])
    return True

# Delete Availability with Local Time Conversion
def delete_availability(email):
    client = authenticate_google_sheets()
    sheet = client.open_by_key(SHEET_ID)
    availability_sheet = sheet.worksheet("Availability")

    # Fetch all availability data
    availabilities = availability_sheet.get_all_records()
    user_availabilities = [a for a in availabilities if a['email'] == email]

    if not user_availabilities:
        print("❌ No availability found for this email.")
        return

    print("Your current availability (converted to local time):")
    for idx, availability in enumerate(user_availabilities):
        utc_datetime_str = f"{availability['date']} {availability['time']}"  # Combine date and time
        local_datetime = convert_utc_to_local(email, utc_datetime_str)
        print(f"{idx + 1}. 📅 Date & Time: {local_datetime}")

    # Prompt to delete availability
    delete_choice = int(input("Choose the availability to delete (1-{}): ".format(len(user_availabilities)))) - 1
    if delete_choice < 0 or delete_choice >= len(user_availabilities):
        print("❌ Invalid choice.")
        return

    row_to_delete = availabilities.index(user_availabilities[delete_choice]) + 2  # Adjust for header row
    availability_sheet.delete_rows(row_to_delete)
    print("✅ Availability deleted successfully.")

# View Availability
def view_availability(email):
    client = authenticate_google_sheets()
    sheet = client.open_by_key(SHEET_ID)
    availability_sheet = sheet.worksheet("Availability")

    # Fetch all availability data
    availabilities = availability_sheet.get_all_records()
    user_availabilities = [a for a in availabilities if a['email'] == email]

    if not user_availabilities:
        print("❌ No availability found for this email.")
        return

    print("Your current availability (converted to local time):")
    for availability in user_availabilities:
        utc_datetime_str = f"{availability['date']} {availability['time']}"  # Combine date and time
        local_datetime = convert_utc_to_local(email, utc_datetime_str)
        print(f"📅 Date & Time: {local_datetime}")

def get_user_roles():
    # Assuming you have a 'Users' sheet that contains user emails and their roles
    client = authenticate_google_sheets()
    sheet = client.open_by_key(SHEET_ID)
    users_sheet = sheet.worksheet("Users")

    # Fetch all user data (assuming it contains 'email' and 'role')
    user_data = users_sheet.get_all_records()

    user_roles = {}
    for user in user_data:
        user_roles[user['email']] = user['role']  # Assuming 'email' and 'role' columns

    return user_roles

def match_users(email):
    client = authenticate_google_sheets()
    sheet = client.open_by_key(SHEET_ID)
    availability_sheet = sheet.worksheet("Availability")
    matched_sheet = sheet.worksheet("Matched")
    users_sheet = sheet.worksheet("Users")

    # Fetch all availability data
    availabilities = availability_sheet.get_all_records()

    # Get user roles from the Users sheet
    user_roles = get_user_roles()  # Fetch roles from Users sheet

    # Get the role of the current user from the user_roles dictionary
    user_role = user_roles.get(email)

    for availability in availabilities:
        if availability['email'] == email:
            date = availability['date']
            time = availability['time']

        # Loop through each user's availability and match them
            for availability in availabilities:
              match_email = availability['email']
              match_date = availability['date']
              match_time = availability['time']
              match_role = user_roles.get(match_email)

              if match_email != email and match_role != user_role and match_date == date and match_time == time:
                  matched_email = match_email

                  # Create Zoom Meeting
                  zoom_link = create_zoom_meeting()
                  case_link = choose_random_case()

                  if zoom_link:
                      print(f"{user_role}")
                      if str(user_role) == 'interviewee':
                        # Store the match in the Matched sheet
                        matched_sheet.append_row([matched_email, email, date, time, zoom_link, case_link])

                        # Remove matched slots from availability
                        remove_matched_slots(availability_sheet, email, matched_email, date, time)

                        # Send email to the Interviewee (Zoom link ONLY)
                        utc_datetime_str = f"{date} {time}"
                        local_datetime = convert_utc_to_local_for_email(email, utc_datetime_str)

                        # Extract local date and time separately
                        local_date_user = local_datetime.strftime("%Y-%m-%d")
                        local_time_user = local_datetime.strftime("%I:%M %p")  # 12-hour format with AM/PM

                        send_email("🎉 You're Matched for an Interview! 🚀",
                                  f"Hello!\n\n"
                                  f"You've been matched with an interviewer! Get ready to shine. ✨\n\n"
                                  f"📅 **Date:** {local_date_user}\n"
                                  f"⏰ **Time:** {local_time_user}\n"
                                  f"🎥 **Zoom Link:** {zoom_link}\n\n"
                                  f"Good luck! 🍀\n"
                                  f"- The CaseBud Team",
                                  email)

                        # Send email to the Interviewer (Zoom link + Case link)
                        utc_datetime_str = f"{date} {time}"
                        local_datetime = convert_utc_to_local_for_email(match_email, utc_datetime_str)

                        # Extract local date and time separately
                        local_date_match = local_datetime.strftime("%Y-%m-%d")
                        local_time_match = local_datetime.strftime("%I:%M %p")  # 12-hour format with AM/PM

                        send_email("🎉 You're Matched for an Interview! 🚀",
                                  f"Hello!\n\n"
                                  f"You've been matched with an interviewee! Time to put your skills to the test. 💡\n\n"
                                  f"📅 **Date:** {local_date_match}\n"
                                  f"⏰ **Time:** {local_time_match}\n"
                                  f"🎥 **Zoom Link:** {zoom_link}\n"
                                  f"📂 **Case Study:** {case_link}\n\n"
                                  f"Good luck! 🍀\n"
                                  f"- The CaseBud Team",
                                  match_email)

                      else:

                        # Store the match in the Matched sheet
                        matched_sheet.append_row([email, matched_email, date, time, zoom_link, case_link])

                        # Remove matched slots from availability
                        remove_matched_slots(availability_sheet, email, matched_email, date, time)

                        # Send email to the Interviewee (Zoom link ONLY)
                        utc_datetime_str = f"{date} {time}"
                        local_datetime = convert_utc_to_local_for_email(match_email, utc_datetime_str)

                        # Extract local date and time separately
                        local_date_match = local_datetime.strftime("%Y-%m-%d")
                        local_time_match = local_datetime.strftime("%I:%M %p")  # 12-hour format with AM/PM

                        send_email("🎉 You're Matched for an Interview! 🚀",
                                  f"Hello!\n\n"
                                  f"You've been matched with an interviewer! Get ready to shine. ✨\n\n"
                                  f"📅 **Date:** {local_date_match}\n"
                                  f"⏰ **Time:** {local_time_match}\n"
                                  f"🎥 **Zoom Link:** {zoom_link}\n\n"
                                  f"Good luck! 🍀\n"
                                  f"- The CaseBud Team",
                                  match_email)

                        # Send email to the Interviewer (Zoom link + Case link)
                        utc_datetime_str = f"{date} {time}"
                        local_datetime = convert_utc_to_local_for_email(email, utc_datetime_str)

                        # Extract local date and time separately
                        local_date_user = local_datetime.strftime("%Y-%m-%d")
                        local_time_user = local_datetime.strftime("%I:%M %p")  # 12-hour format with AM/PM

                        send_email("🎉 You're Matched for an Interview! 🚀",
                                  f"Hello!\n\n"
                                  f"You've been matched with an interviewee! Time to put your skills to the test. 💡\n\n"
                                  f"📅 **Date:** {local_date_user}\n"
                                  f"⏰ **Time:** {local_time_user}\n"
                                  f"🎥 **Zoom Link:** {zoom_link}\n"
                                  f"📂 **Case Study:** {case_link}\n\n"
                                  f"Good luck! 🍀\n"
                                  f"- The CaseBud Team",
                                  email)

                      #Swap roles
                      swap_roles(email, matched_email, user_role, match_role)

            else:
              utc_datetime_str = f"{date} {time}"
              local_datetime = convert_utc_to_local_for_email(email, utc_datetime_str)
              local_date_user = local_datetime.strftime("%Y-%m-%d")
              local_time_user = local_datetime.strftime("%I:%M %p")  # 12-hour format with AM/PM
              print(f"An email will be sent to you when a match is found for {local_date_user} {local_time_user}")

            # Ensure we move to the next availability entry after creating a match
            continue

def swap_roles(email1, email2, userrole1, userrole2):
    client = authenticate_google_sheets()
    sheet = client.open_by_key(SHEET_ID)
    users_sheet = sheet.worksheet("Users")

    # Fetch all user data (assuming it contains 'email' and 'role')
    user_data = users_sheet.get_all_records()

    # Convert dict_keys to a list before indexing
    role_column_index = list(user_data[0].keys()).index('role') + 1

    for idx, user in enumerate(user_data):
        if user['email'] == email1:
            users_sheet.update_cell(idx + 2, role_column_index, userrole2)
        elif user['email'] == email2:
            users_sheet.update_cell(idx + 2, role_column_index, userrole1)

def remove_matched_slots(sheet, email1, email2, date, time):
    # Find the availability entries for both users and remove them
    availabilities = sheet.get_all_records()

    for availability in availabilities:
        if availability['email'] == email1 and availability['date'] == date and availability['time'] == time:
            sheet.delete_rows(availabilities.index(availability) + 2)  # +2 to adjust for row indexing
            break

    for availability in availabilities:
        if availability['email'] == email2 and availability['date'] == date and availability['time'] == time:
            sheet.delete_rows(availabilities.index(availability) + 2)  # +2 to adjust for row indexing
            break

def login(email, password):
    client = authenticate_google_sheets()
    sheet = client.open_by_key(SHEET_ID)
    users_sheet = sheet.worksheet("Users")
    user_records = users_sheet.get_all_values()  # Fetch all user records
    for row in user_records:
        if row[0] == email:  # Email stored in the first column
            stored_hashed_password = row[1]  # Password stored in the second column
            if bcrypt.checkpw(password.encode(), stored_hashed_password.encode()):
                return True
            else:
                print("❌ Incorrect password.")
                return False
    print("❌ Account not found.")
    return False

# Main Program Flow
def main():
    print("Welcome to CaseBud Matching!")

    while True:
        print("\nChoose an option:")
        print("1. Login")
        print("2. Create Account")
        print("3. Exit")

        option = input("Select an option (1-3): ")

        if option == "1":
            email = input("Please enter your email to login: ")
            password = getpass.getpass(prompt="Enter your password: ")

            if login(email, password):
                print("✅ Logged in successfully.")
                account_menu(email)
            else:
                print("❌ Invalid credentials.")

        elif option == "2":
            email = input("Please enter your email to create an account: ")
            password = getpass.getpass(prompt="Enter your password: ")

            if create_account(email, password):
                print("✅ Account created successfully.")
                account_menu(email)

        elif option == "3":
            print("Exiting...")
            break

        else:
            print("❌ Invalid choice. Please try again.")

def account_menu(email):
    while True:
        print("\nChoose an option:")
        print("1. Add Availability")
        print("2. Delete Availability")
        print("3. View Availability")
        print("4. Exit")

        option = input("Select an option (1-4): ")

        if option == "1":
            add_availability(email)
        elif option == "2":
            delete_availability(email)
        elif option == "3":
            view_availability(email)
        elif option == "4":
            print("Exiting...")
            break
        else:
            print("❌ Invalid choice. Please try again.")

if __name__ == "__main__":
    main()


Welcome to CaseBud Matching!

Choose an option:
1. Login
2. Create Account
3. Exit
Select an option (1-3): 1
Please enter your email to login: avleenkchawla@gmail.com
Enter your password: ··········
✅ Logged in successfully.

Choose an option:
1. Add Availability
2. Delete Availability
3. View Availability
4. Exit
Select an option (1-4): 4
Exiting...

Choose an option:
1. Login
2. Create Account
3. Exit
Select an option (1-3): 1
Please enter your email to login: avleenkchawla@gmail.com
Enter your password: ··········
✅ Logged in successfully.

Choose an option:
1. Add Availability
2. Delete Availability
3. View Availability
4. Exit
Select an option (1-4): 1
1. Monday, 2025-03-10
2. Tuesday, 2025-03-11
3. Wednesday, 2025-03-12
4. Thursday, 2025-03-13
5. Friday, 2025-03-14
6. Saturday, 2025-03-15
7. Sunday, 2025-03-16
Choose the date (1-7): 3
Choose time slots between 8AM - 9PM, comma-separated (e.g., 9, 10, 14):
Enter available hours in 24-hour format (e.g., 9, 10, 14): 15
✅ Availabilit